# 第10章 並行プログラム

## 10.2 宣言型の並行処理

要件: 都市ランキング

1. このプログラムでは、世界中の人々のチェックインから成るストリームを処理する必要がある(Stream[IO, City]型の値が提供される)
2. このプログラムでは、チェックインがまだ処理中であっても、(チェックインでランク付けされた)現在の上位3都市のランキングを取得できなければならない。

In [19]:
object model {
  opaque type City = String
  object City {
    def apply(name: String): City = name
    extension (city: City) def name: String = city
  }
  case class CityStats(city: City, checkIns: Int)
}
import model._


defined object model
import model._



## 10.3 逐次と並行

学習プロセス:

1. 逐次IO
2. ファイバを使ったIO
3. 同時IOと非同期アクセス


## 10.4 コーヒーブレイク: 逐次的に考える

In [20]:
import $ivy.`org.typelevel::cats-effect:3.2.9`
import $ivy.`co.fs2::fs2-core:3.1.2`

import cats.effect._
import fs2.Stream

// チェックインを1つずつ処理し、各チェックイン要素を処理した後に現在のランキングを生成する関数を実装する。
def processCheckIns(checkIns: Stream[IO, City]): IO[Unit] = ???

/**
 * ヒント:
 * scan...foldLeftみたいなやつ
 * foreach...コレクションの各要素に対して操作を行う。コレクションの各要素に対して引数として渡された関数を適用する。値は返されない。
 * Map...ハッシュマップ
 * updated(またはupdatedWith)
 */


import $ivy.$                                 

import $ivy.$                       


import cats.effect._

import fs2.Stream

// チェックインを1つずつ処理し、各チェックイン要素を処理した後に現在のランキングを生成する関数を実装する。

defined function processCheckIns

In [21]:
// 上位3都市のランキング
def topCities(cityCheckIns: Map[City, Int]): List[CityStats] = {
  cityCheckIns.toList
    .map(_ match {
      case (city, checkIns) => CityStats(city, checkIns)
    })
    .sortBy(_.checkIns)
    .reverse
    .take(3)
}

def processCheckIns(checkIns: Stream[IO, City]): IO[Unit] = {
  checkIns
    .scan(Map.empty[City, Int])((cityCheckIns, city) =>
      cityCheckIns
        .updatedWith(city)(_.map(_ + 1).orElse(Some(1)))
    )
    .map(topCities)
    .foreach(IO.println)
    .compile.drain
}


defined function topCities
defined function processCheckIns

In [22]:
import cats.effect.unsafe.implicits.global

// 実行例
val checkIns: Stream[IO, City] =
  Stream(
    City("Sydney"),
    City("Sydney"),
    City("Cape Town"),
    City("Singapore"),
    City("Cape Town"),
    City("Sydney")
  ).covary[IO]

// processCheckIns関数を呼び出すと、プログラムが返される。
// そのプログラムを実行すると、ランキングの更新情報が7つコンソールに出力され
// (最初の空のランキングを含む)、Unit型の値(())が返される。
processCheckIns(checkIns).unsafeRunSync()
/**
 * List()
 * List(CityStats(City(Sydney), 1))
 * List(CityStats(City(Sydney), 2))
 * List(CityStats(City(Sydney), 2), CityStats(City(Cape Town), 1))
 * ...
 */


List()
List(CityStats(Sydney,1))
List(CityStats(Sydney,2))
List(CityStats(Sydney,2), CityStats(Cape Town,1))
List(CityStats(Sydney,2), CityStats(Singapore,1), CityStats(Cape Town,1))
List(CityStats(Cape Town,2), CityStats(Sydney,2), CityStats(Singapore,1))
List(CityStats(Sydney,3), CityStats(Cape Town,2), CityStats(Singapore,1))


import cats.effect.unsafe.implicits.global

// 実行例

checkIns: Stream[_root_.fs2.Stream[[A >: scala.Nothing <: scala.Any] => _root_.cats.effect.IO[A], _root_.ammonite.$sess.cell19.instance.model.City], City] = Stream(..)

## 10.6 バッチ処理の必要性

先程作成したコードでは都市の数が増えるほどソートプロセスに時間がかかるようになる。

In [23]:
// 大規模なチェックインストリーム
val checkIns: Stream[IO, City] =
  Stream(City("Sydney"), City("Dublin"), City("Cape Town"), City("Lima"), City("Singapore"))
    .repeatN(100_000)
    .append(Stream.range(0, 100_000).map(i => City(s"City $i")))
    .append(Stream(City("Sydney"), City("Sydney"), City("Lima")))
    .covary[IO]

// めっちゃ時間かかる
// processCheckIns(checkIns).unsafeRunSync()


checkIns: Stream[_root_.fs2.Stream[[A >: scala.Nothing <: scala.Any] => _root_.cats.effect.IO[A], _root_.ammonite.$sess.cell19.instance.model.City], City] = Stream(..)

## 10.7 バッチ処理を実装する

chunkN...数値nを受け取り、n個の要素を1つのコレクションのような要素に変換した上で出力する

In [24]:
def processCheckIns(checkIns: Stream[IO, City]): IO[Unit] = {
  checkIns
    .scan(Map.empty[City, Int])((cityCheckIns, city) =>
      cityCheckIns
        .updatedWith(city)(_.map(_ + 1).orElse(Some(1)))
    )
    .chunkN(100_000)
    .map(_.last)
    .unNone
    .map(topCities)
    .foreach(IO.println)
    .compile.drain
}

processCheckIns(checkIns).unsafeRunSync()


List(CityStats(Sydney,20000), CityStats(Lima,20000), CityStats(Dublin,20000))
List(CityStats(Sydney,40000), CityStats(Lima,40000), CityStats(Dublin,40000))
List(CityStats(Sydney,60000), CityStats(Lima,60000), CityStats(Dublin,60000))
List(CityStats(Sydney,80000), CityStats(Lima,80000), CityStats(Dublin,80000))
List(CityStats(Sydney,100000), CityStats(Lima,100000), CityStats(Dublin,100000))
List(CityStats(Singapore,100000), CityStats(Sydney,100000), CityStats(Lima,100000))
List(CityStats(Sydney,100002), CityStats(Lima,100001), CityStats(Singapore,100000))


defined function processCheckIns

## 10.8 並行処理の世界

本章ではスレッドを2つ使う。  
1つはチェックイン用、もう1つはランキング用。  
ただし、スレッドの数がもっと増えても安全性が確保されるような方法で実装する。

## 10.12 Refの登場

> 並行プログラミを作成するにあたって、ロックを作成したり、ロックを待機したり、  
> 処理が終わったときに他のスレッドに通知したりする必要はない。  
> また、ミュータブルなデータ構造で妥協する必要もない。  
> 関数型プログラミングでは、あらゆるものをイミュータブルな値としてモデル化する。
>
> **Ref[IO, A]** はイミュータブルな値であり、型Aのイミュータブルな値に対する参照を表す。  
> この参照は、**同時にアクセスすることが可能な、非同期のミュータブルな参照である**。  
> 要するにAtomicReferenceのラッパーであり、ミューバルな参照であるため、副作用がある。

## 10.13 Ref型の値を更新する

Ref[IO, A]には、次のシグネチャを持つupdate関数がある。

```scala
def update(f: A => A): IO[Unit]
```

update関数はIO型の値を返す。つまり副作用のあるプログラムのディスクリプションが返される。

## 10.14 Ref型の値を使う

```scala
// Ref.of関数のシグネチャ
def of(a: A): IO[Ref[IO, A]]

// Ref型の値を作成するには
Ref.of[IO, A](initialValue: A)

// Ref型の値を取得する関数Ref.getのシグネチャ
def get: IO[A]

// 例:
val example: IO[Int] = for {
  counter <- Ref.of[IO, Int](0)
  _       <- counter.update(_ + 3)
  result  <- counter.get
} yield result

example.unsafeRunSync()
// →3
```

## 10.15 すべてを同時に行う

関数型プログラミングの並列処理では、Threadオブジェクトを作成する必要はない。  
**並列に実行すべきものを宣言するだけでよい**。  

> ある意味、並行アプリケーションは小さな、副作用のある、逐次プログラムの集まりにすぎない。  
> それらのプログラムが並列に実行されるというだけのことだ。

- 逐次プログラム

```scala
val exampleSequential: IO[Int] = for {
  counter <- Ref.of[IO, Int](0)
  _       <- List(counter.update(_ + 2), counter.update(_ + 3), counter.update(_ + 4)).sequence
  result  <- counter.get
} yield result

exampleSequential.unsafeRunSync()
// →9
```

IO型の値からなるListで定義されているsequence関数は、それらの値を順番に実行する。  
sequenceの並列処理バージョンがparSequenceである。

- 並列プログラム

```scala
val exampleConcurrent: IO[Int] = for {
  counter <- Ref.of[IO, Int](0)
  _       <- List(counter.update(_ + 2), counter.update(_ + 3), counter.update(_ + 4)).parSequence    // ！
  result  <- counter.get
} yield result

exampleConcurrent.unsafeRunSync()
// →9
```

## 10.16 parSequenceを使う

- IO.sleep

```scala
def sleep(delay: FiniteDuration): IO[Unit]
```

この関数はプログラムを表すIO型の値を返す。そのプログラムを実行すると、  
指定された時間だけスリープして制御を戻す。  
いわばThread.sleepの純粋関数型バージョンである。この関数はスレッドをブロックしない。  
flatMapを使うと逐次プログラムが得られる。

```scala
// 最初に1秒間スリープした後、Ref型の値に3を足して更新するプログラム
IO.sleep(FiniteDuration(1, TImeUnit.SECONDS)).flatMap(_ => counter.update(_ + 3))
```

```scala
for {
  counter  <- Ref.of[IO, Int](0)
  program1 = counter.update(_ + 2)
  program2 = IO.sleep(FiniteDuration(1, TImeUnit.SECONDS)).flatMap(_ => counter.update(_ + 3))
  program3 = IO.sleep(FiniteDuration(1, TImeUnit.SECONDS)).flatMap(_ => counter.update(_ + 4))

  _        <- List(program1, program2, program3).sequence
  // _        <- List(program1, program2, program3).parSequence
  result  <- counter.get
} yield result
```



## 10.17 実習: 同時IO

1. 1秒間待機してから2つのサイコロを振り、両方が終わるのを待ってそれらの目の合計を返す。
2. 2つのサイコロを同時に振り、それぞれの結果を同時アクセスが可能な参照(List)に格納し、その値を結果として返す。
3. 3つのサイコロを同時に振り、それぞれの結果を同時アクセスが可能な参照(List)に格納し、その値を結果として返す。
4. 100個のサイコロを同時に振り、6の目の合計数を同時アクセスが可能な参照に格納し、その値を結果として返す。
5. 100個のサイコロを同時に振り、それぞれの前に1秒間待機し、それらの合計を(同時参照を使わずに)返す。


In [25]:
import cats.effect._
import scala.concurrent._
import scala.concurrent.duration._
import cats.implicits._

// サイコロ関数
def castTheDie(): IO[Int] = ???

// 1. 1秒間待機してから2つのサイコロを振り、両方が終わるのを待ってそれらの目の合計を返す。
for {
  _      <- IO.sleep(1.second)
  result <- List(castTheDie(), castTheDie()).parSequence
} yield result.sum

// 2. 2つのサイコロを同時に振り、それぞれの結果を同時アクセスが可能な参照(List)に格納し、その値を結果として返す。
for {
  storedCasts <- Ref.of[IO, List[Int]](List.empty)
  singleCast  <- castTheDie().map(result => storedCasts.update(_.appended(result)))
  // flatMapを使うとコンパイルエラーになる
  // singleCast  <- castTheDie().flatMap(result => storedCasts.update(_.appended(result)))
  _           <- List(singleCast, singleCast).parSequence
  casts       <- storedCasts.get
} yield casts

// 3. 3つのサイコロを同時に振り、それぞれの結果を同時アクセスが可能な参照(List)に格納し、その値を結果として返す。
for {
  storedCasts <- Ref.of[IO, List[Int]](List.empty)
  singleCast  <- castTheDie().map(result => storedCasts.update(_.appended(result)))
  // flatMapを使うとコンパイルエラーになる
  // singleCast  <- castTheDie().flatMap(result => storedCasts.update(_.appended(result)))
  _           <- List(singleCast, singleCast, singleCast).parSequence
  casts       <- storedCasts.get
} yield casts

// 4. 100個のサイコロを同時に振り、6の目の合計数を同時アクセスが可能な参照に格納し、その値を結果として返す。
for {
  storedCasts <- Ref.of[IO, Int](0)
  singleCast  <- castTheDie()
                  .map(result =>
                    if (result == 6) storedCasts.update(_ + 1) else IO.unit)
  _           <- List.fill(100)(singleCast).parSequence
  casts       <- storedCasts.get
} yield casts

// 5. 100個のサイコロを同時に振り、それぞれの前に1秒間待機し、それらの合計を(同時参照を使わずに)返す。
List.fill(100)(IO.sleep(1.second).flatMap(_ => castTheDie())).parSequence.map(_.sum)


import cats.effect._

import scala.concurrent._

import scala.concurrent.duration._

import cats.implicits._

// サイコロ関数

defined function castTheDie
res25_5: IO[Int] = FlatMap(
  ioe = Sleep(delay = 1 second),
  f = ammonite.$sess.cell25$Helper$$Lambda$4357/0x00000008017c7330@25686b59,
  event = cats.effect.tracing.TracingEvent$StackTrace
)
res25_6: IO[List[Int]] = FlatMap(
  ioe = Delay(
    thunk = cats.effect.IO$$$Lambda$3773/0x00000008016c2620@1017363b,
    event = cats.effect.tracing.TracingEvent$StackTrace
  ),
  f = ammonite.$sess.cell25$Helper$$Lambda$4358/0x00000008017c76f8@24af291f,
  event = cats.effect.tracing.TracingEvent$StackTrace
)
res25_7: IO[List[Int]] = FlatMap(
  ioe = Delay(
    thunk = cats.effect.IO$$$Lambda$3773/0x00000008016c2620@165607f0,
    event = cats.effect.tracing.TracingEvent$StackTrace
  ),
  f = ammonite.$sess.cell25$Helper$$Lambda$4359/0x00000008017c7ac0@30e0332,
  event = cats.effect.tracing.TracingEvent$StackTrace
)
res25_8: IO[Int] = FlatMap(
  io

## 10.18 平行性をモデル化する

同時参照が2つ必要:

1. チェックインを格納するための参照(storedCheckIns)
2. 現在のランキングを格納するための参照(storedRanking)

よって少なくとも2種類の逐次プログラムが必要になる。

1. checkInsProgramは、実行時にチェックインからなる入力ストリームをdrainし、それぞれをstoredCheckIns参照に安全に格納する。
2. rankingProgramは、実行時に現在のチェックインを無限に読み取り、ランキングを計算し、storedRanking参照に格納する。

最終的なソリューションでは、両方のプログラムを同時に実行する必要がある。

## 10.19 Refとファイバを使ったコーディング

In [26]:
// processCheckInsを並行処理を行うようにアップデート
def processCheckIns(checkIns: Stream[IO, City]): IO[Unit] = {
  for {
    // チェックイン、ランキングを格納する同時参照を実装
    storedCheckIns  <- Ref.of[IO, Map[City, Int]](Map.empty)
    storedRanking   <- Ref.of[IO, List[CityStats]](List.empty)
    // 同時参照を更新するプログラムを実装
    rankingProgram  = updateRanking(storedCheckIns, storedRanking)
    checkInsProgram = checkIns.evalMap(storeCheckIn(storedCheckIns)).compile.drain
    // 2つのプログラムを並行処理
    _               <- List(rankingProgram, checkInsProgram).parSequence
  } yield ()
}

// ランキング更新プログラムのシグネチャ
def updateRanking(storedCheckIns: Ref[IO, Map[City, Int]], storedRanking: Ref[IO, List[CityStats]]): IO[Unit] = ???

// チェックイン更新プログラム
def storeCheckIn(storedCheckIns: Ref[IO, Map[City, Int]])(city: City): IO[Unit] = {
  storedCheckIns.update(_.updatedWith(city)(_ match {
    case None           => Some(1)
    case Some(checkIns) => Some(checkIns + 1)
  }))
}


defined function processCheckIns
defined function updateRanking
defined function storeCheckIn

未解決の問題:

- updateRanking関数の実装が必要。
- processCheckIns関数から返されたIO型の値を実行しても、フィードバックは返されない。  
  ユーザーのために現在のランキングを出力する方法はまだ提供されていない。  
  単純なprintlnすら使っていない。

## 10.20 決して終わらないIO

Notingは値を持たない型で、IO[Nothing]は実行時にプログラムから制御が戻らない、  
またはプログラムが失敗することを意味する。他の関数型プログラミング言語では  
Nothing型を「ボトム型」と呼ぶ。

In [27]:
// Nothing型を使ってupdateRankingを実装
def updateRanking(storedCheckIns: Ref[IO, Map[City, Int]], storedRanking: Ref[IO, List[CityStats]]): IO[Nothing] = {
  for {
    newRanking <- storedCheckIns.get.map(topCities)
    _          <- storedRanking.set(newRanking)
    result     <- updateRanking(storedCheckIns, storedRanking)
  } yield result
}

// updateRankingの別バージョン: foreverMを使う
def updateRanking2(storedCheckIns: Ref[IO, Map[City, Int]], storedRanking: Ref[IO, List[CityStats]]): IO[Nothing] = {
  (for {
    newRanking <- storedCheckIns.get.map(topCities)
    _          <- storedRanking.set(newRanking)
  } yield ()).foreverM
}

// updateRankingの別バージョン: for内包表記を使わずflatMap/mapを使う
def updateRanking3(storedCheckIns: Ref[IO, Map[City, Int]], storedRanking: Ref[IO, List[CityStats]]): IO[Nothing] = {
  storedCheckIns.get.map(topCities).flatMap(storedRanking.set).foreverM
}


defined function updateRanking
defined function updateRanking2
defined function updateRanking3

## 10.21 コーヒーブレイク: 並行的に考える

要件(おさらい): 都市ランキング

1. このプログラムでは、世界中の人々のチェックインから成るストリームを処理する必要がある(Stream[IO, City]型の値が提供される)
2. このプログラムでは、チェックインがまだ処理中であっても、(チェックインでランク付けされた)現在の上位3都市のランキングを取得できなければならない。

最新のランキングを確認できるようにする。  
その前に、復習として、processCheckInsを更新し、**現在のランキングを1秒おきに出力する**ようにしてみる。

In [28]:
// 1秒毎にランキングを出力する
def outputRankingPerSec(storedRanking: Ref[IO, List[CityStats]]): IO[Nothing] = {
  for {
    _       <- IO.sleep(1.second)
    ranking <- storedRanking.get
    _       <- IO.println(ranking)
    result  <- outputRankingPerSec(storedRanking)
  } yield result
}

// 現在のランキングを1秒おきに出力するようprocessCheckInsをアップデート
def processCheckIns(checkIns: Stream[IO, City]): IO[Unit] = {
  for {
    storedCheckIns  <- Ref.of[IO, Map[City, Int]](Map.empty)
    storedRanking   <- Ref.of[IO, List[CityStats]](List.empty)
    rankingProgram  = updateRanking(storedCheckIns, storedRanking)
    // 個人の回答
    outputProgram   = outputRankingPerSec(storedRanking)
    // 書籍の回答
    // outputProgram   = IO.sleep(1.second)
    //                     .flatMap(_ => storedRanking.get)
    //                     .flatMap(IO.println)
    //                     .foreverM
    checkInsProgram = checkIns.evalMap(storeCheckIn(storedCheckIns)).compile.drain
    _               <- List(rankingProgram, checkInsProgram, outputProgram).parSequence
  } yield ()
}


defined function outputRankingPerSec
defined function processCheckIns

## 10.23 非同期性のニーズ

要件は「すでに処理されているチェックインの数にかかわらず、最新のランキングにいつでもアクセスできるようにしたい」とある。  
チェックインの処理とランキングの生成を完全に切り離すために、スレッドを**非同期で**開始する必要がある。

## 10.24 非同期アクセスの準備

現在のプログラムは並行プログラムだが、**同期的**に使っている。  
つまり、プログラムを実行したら、**プログラムが完了するまで待つ**必要がある。  
結果を待っている間、**呼び出し元のスレッドはブロックされる**。

現在の同期ソリューションを非同期ソリューションに置き換える。

## 10.25 関数型非同期プログラムを設計する

> プログラムを返す関数が必要である。そのプログラムは実行時にすべてのスレッドを生成し、  
> 同時参照を使ってそれらのスレッドを接続し、直ちに制御を戻して、現在のランキングにいつでもアクセスできるようにする。  
> 関数型プログラミングでは、すべてのものをイミュータブルな値としてモデル化するため、  
> そのような非同期ハンドルもイミュータブルな値としてモデル化できる。  
> このハンドルをProcessCheckInsと呼ぶことにする。

In [29]:
case class ProcessingCheckIns(
  currentRanking: IO[List[CityStats]],
  stop: IO[Unit]
)


defined class ProcessingCheckIns

この直積型を戻り値の型の一部として使うことができる。

In [30]:
def processCheckIns(checkIns: Stream[IO, City]): IO[ProcessingCheckIns] = ???


defined function processCheckIns

新しいバージョンを実装する前に、そのようなAPIを持つ関数を使って非同期アクセスを実現する方法を確認する。

```scala
for {
  processing <- processCheckIns(checkIns)
  ranking    <- processing.currentRanking
  _          <- IO.println(ranking)
  _          <- IO.sleep(1.second)

  // 他の処理

  newRanking <- processing.currentRanking
  _          <- processing.stop
} yield newRanking
```

> これはクライアントが作成できるであろうプログラムの例である。  
> まず、processCheckIns関数を呼び出す必要がある。この関数はすべてのファイバを生成し、  
> IO型の2つの値が含まれたProcessingCheckIns型の値を返す。  
> 値の1つは現在のランキングを返し、もう1つはすべてのファイバを停止する。  
> つまり、両方の値を都合のよいタイミングで実行する責任はクライアントにある。

## 10.26 ファイバを手動で管理する

IO型にはstart関数がある。この関数はIO型の値をファイバで実行し、このファイバへのハンドルをその終了を待たずに返すことができる。

```scala
// IO.start関数のシグネチャ
def start[A]: IO[FiberIO[A]]
```

ファイバへのハンドルは戻り値を見てわかる通りFiberIO[A]型のイミュータブルな値で表される。  
このハンドルを使ってさまざまなことができるが、最も重要なのはcancel関数である。

```scala
// FiberIO[A].cancel関数のシグネチャ
def cancel: IO[Unit]
```

このハンドルには、IO[Unit]を返すcancel関数がある。  
IO[Unit]は実行時に元のファイバをキャンセルするプログラムである。

ただし、同時IOの作成に関しては、startやcancelのような関数よりも、parSequence関数  
(および同様のセマンティクスを持つ他の関数)を使うほうがよい選択肢と見なされる。(p.394)

## 10.27 関数型の非同期プログラムのコーディング

start関数、cancel関数を使って、processCheckInsの最終バージョンをモデル化する。

In [32]:
// processCheckInsの最終バージョン
def processCheckIns(checkIns: Stream[IO, City]): IO[ProcessingCheckIns] = {
  for {
    storedCheckIns  <- Ref.of[IO, Map[City, Int]](Map.empty)
    storedRanking   <- Ref.of[IO, List[CityStats]](List.empty)
    rankingProgram  = updateRanking(storedCheckIns, storedRanking)
    checkInsProgram = checkIns.evalMap(storeCheckIn(storedCheckIns)).compile.drain
    fiber           <- List(rankingProgram, checkInsProgram).parSequence.start
  } yield ProcessingCheckIns(storedRanking.get, fiber.cancel)
}


defined function processCheckIns

## 10.28 本章のまとめ

- 並行プログラムのフローを宣言型で設計する
- 軽量な仮想スレッド(ファイバ)を使う
- さまざまなスレッドのデータを安全に格納し、それらのデータにアクセスする
- イベントのストリームを非同期で処理する